In [ ]:
import os
from keras.models import Model
from keras.layers import Input, Dense, ReLU, Dropout, Activation, concatenate, Softmax, Conv2D, MaxPool2D, Add, BatchNormalization
from keras.layers import Bidirectional, Flatten, CuDNNGRU
from keras.utils.vis_utils import plot_model
import keras

# ===========================
# Define a custom ResNet-like CNN model
# ===========================
def ResNet(weights=None,
           input_shape=[2,1024],   # Input shape (e.g., 2x1024)
           classes=26,             # Number of output classes for classification
           **kwargs):

    # Check if weights file exists if provided
    if weights is not None and not (os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), '
                         'or the path to the weights file to be loaded.')

    dr = 0.6  # Dropout rate for regularization

    # Input layer
    input = Input(input_shape + [1], name='input')  # Add channel dimension for Conv2D

    # --------------------------
    # First convolutional layer
    # --------------------------
    x = Conv2D(256, (1,3), name="conv1", kernel_initializer='glorot_uniform', padding='same')(input)
    x = Activation('relu')(x)
    # x = Dropout(dr)(x)  # Dropout optional, commented out

    # --------------------------
    # Second convolutional layer
    # --------------------------
    x = Conv2D(256, (2,3), name="conv2", kernel_initializer='glorot_uniform', padding='same')(x)
    # x = Dropout(dr)(x)  # Dropout optional

    # --------------------------
    # Residual connection: add input to conv output
    # --------------------------
    x1 = Add()([input, x])
    x1 = Activation('relu')(x1)

    # --------------------------
    # Third and fourth convolutional layers
    # --------------------------
    x = Conv2D(80, (1,3), activation="relu", name="conv3", kernel_initializer='glorot_uniform', padding='same')(x1)
    x = Conv2D(80, (1,3), activation="relu", name="conv4", kernel_initializer='glorot_uniform', padding='same')(x)
    x = Dropout(dr)(x)  # Apply dropout for regularization

    # --------------------------
    # Flatten and fully connected layers
    # --------------------------
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='fc1')(x)
    x = Dropout(dr)(x)

    # --------------------------
    # Output layer with softmax activation for multi-class classification
    # --------------------------
    output = Dense(classes, activation='softmax', name='softmax')(x)

    # Create Keras model
    model = Model(inputs=input, outputs=output)

    # Load pre-trained weights if provided
    if weights is not None:
        model.load_weights(weights)

    return model


# ===========================
# Main script to create, compile, and inspect the model
# ===========================
if __name__ == '__main__':
    # Instantiate the model with specific input and output size
    model = ResNet(None, input_shape=[2,128], classes=11)

    # Define Adam optimizer with custom parameters
    adam = keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=None,
        decay=0.0,
        amsgrad=False
    )

    # Compile the model
    # - Loss: categorical_crossentropy (for multi-class classification)
    # - Metrics: accuracy
    # - Optimizer: Adam
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

    # Print model information
    print('Model layers:', model.layers)       # List of layers in the model
    print('Model config:', model.get_config()) # Configuration details of each layer
    print('Model summary:')
    model.summary()                            # Detailed summary with output shapes and parameters

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend for matplotlib (needed on some systems)
import matplotlib.pyplot as plt
import numpy as np
import pickle
import csv
import itertools
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error, mean_absolute_error, r2_score

# ===========================
# Function: plot_confusion_matrix
# Visualizes a confusion matrix
# ===========================
def plot_confusion_matrix(cm, title='', cmap=plt.get_cmap("Blues"), labels=[], save_filename=None):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm*100, interpolation='nearest', cmap=cmap)  # Multiply by 100 for percentage
    plt.title(title, fontsize=10)
    plt.colorbar()

    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=90, size=12)  # X-axis labels
    plt.yticks(tick_marks, labels, size=12)               # Y-axis labels

    # Annotate each cell with the value
    for i in range(len(tick_marks)):
        for j in range(len(tick_marks)):
            if i != j:
                plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10)
            else:  # Diagonal (correct predictions)
                color = 'darkorange'  # Highlight diagonal cells
                plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10, color=color)

    plt.tight_layout()
    plt.ylabel('True label', fontdict={'size':16,})
    plt.xlabel('Predicted label', fontdict={'size':16,})

    # Save figure if filename is provided
    if save_filename is not None:
        plt.savefig(save_filename, format='pdf', dpi=1200, bbox_inches='tight')
    plt.close()

# ===========================
# Function: calculate_confusion_matrix
# Computes confusion matrix and counts correct/incorrect predictions
# ===========================
def calculate_confusion_matrix(Y, Y_hat, classes):
    n_classes = len(classes)
    conf = np.zeros([n_classes, n_classes])      # Raw counts of predictions
    confnorm = np.zeros([n_classes, n_classes])  # Normalized per-class confusion

    # Fill confusion matrix
    for k in range(Y.shape[0]):
        i = list(Y[k,:]).index(1)           # True label index (one-hot to integer)
        j = int(np.argmax(Y_hat[k,:]))      # Predicted label index
        conf[i,j] += 1

    # Normalize confusion matrix by row
    for i in range(n_classes):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

    right = np.sum(np.diag(conf))  # Total correct predictions
    wrong = np.sum(conf) - right   # Total incorrect predictions
    return confnorm, right, wrong

# ===========================
# Function: calculate_acc_at1snr_from_cm
# Computes per-class accuracy from normalized confusion matrix
# ===========================
def calculate_acc_at1snr_from_cm(cm):
    return np.round(np.diag(cm) / np.sum(cm, axis=1), 3)  # Diagonal / row sum

# ===========================
# Function: calculate_metrics
# Computes overall metrics (accuracy, precision, recall, F1, etc.)
# ===========================
def calculate_metrics(Y, Y_hat):
    Y_true = np.argmax(Y, axis=1)  # Convert one-hot labels to integers
    Y_pred = np.argmax(Y_hat, axis=1)

    # Compute metrics
    accuracy = accuracy_score(Y_true, Y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(Y_true, Y_pred, average='weighted')
    mse = mean_squared_error(Y_true, Y_pred)
    mae = mean_absolute_error(Y_true, Y_pred)
    r2 = r2_score(Y_true, Y_pred)

    return accuracy, precision, recall, f1

# ===========================
# Function: calculate_acc_cm_each_snr
# Calculate and plot metrics at each SNR level
# ===========================
def calculate_acc_cm_each_snr(Y, Y_hat, Z, classes=None, save_figure=True, min_snr=0):
    Z_array = Z[:, 0]                        # Extract SNR values from Z
    snrs = sorted(list(set(Z_array)))        # Unique SNR levels
    acc = np.zeros(len(snrs))                # Accuracy for each SNR
    acc_mod_snr = np.zeros((len(classes), len(snrs)))  # Per-class accuracy per SNR

    # Store metrics for plotting
    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }

    i = 0
    for snr in snrs:
        # Select samples corresponding to current SNR
        Y_snr = Y[np.where(Z_array == snr)]
        Y_hat_snr = Y_hat[np.where(Z_array == snr)]

        # Calculate metrics
        accuracy, precision, recall, f1 = calculate_metrics(Y_snr, Y_hat_snr)

        # Append metrics to dictionary
        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)

    # --------------------------
    # Plot metrics vs SNR
    # --------------------------
    for metric, values in metrics.items():
        plt.figure(figsize=(8, 6))
        plt.plot(snrs, values, label=metric)
        # Annotate values on plot
        for x, y in zip(snrs, values):
            plt.text(x, y, round(y, 3), ha='center', va='bottom', fontsize=8)
        plt.xlabel("Signal to Noise Ratio (dB)")
        plt.ylabel(metric.capitalize())
        plt.title(f"{metric.capitalize()} vs SNR")
        plt.legend()
        plt.grid()
        # Save figure as PDF
        plt.savefig(f'figure/{metric}_vs_snr.pdf', format='pdf', dpi=1200, bbox_inches='tight')
        plt.show()

In [ ]:
import os, random

# ===========================
# Configure Keras and GPU settings
# ===========================
os.environ["KERAS_BACKEND"] = "tensorflow"       # Set Keras backend to TensorFlow
# os.environ["THEANO_FLAGS"] = "device=gpu%d"%(0) # Optional: If using Theano backend, select GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "1"        # Specify which GPU to use (GPU 1)

import numpy as np
import matplotlib
# matplotlib.use('Tkagg')  # Optional: Use TkAgg backend for matplotlib if needed
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
# from matplotlib import pyplot as plt  # Alternative import (commented out)
import pickle, sys, h5py
import keras
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras.regularizers import *
from keras.optimizers import Adam
from keras.models import model_from_json
# from keras.utils.vis_utils import plot_model  # Optional: visualize model structure

# ===========================
# TensorFlow and Pandas
# ===========================
import tensorflow as tf
import pandas as pd

# ===========================
# Keras utility for converting labels to one-hot encoding
# ===========================
from keras.utils.np_utils import to_categorical

# ===========================
# Define the modulation classes for classification
# ===========================
classes = [
    'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '64PSK',
    '4QAM', '8QAM', '16QAM', '32QAM', '64QAM', '128QAM', '256QAM',
    '2FSK', '4FSK', '8FSK', '16FSK',
    '4PAM', '8PAM', '16PAM',
    'AM-DSB', 'AM-DSB-SC', 'AM-USB', 'AM-LSB',
    'FM', 'PM'
]
# This list represents all the modulation schemes that the model will classify.
# It is used for one-hot encoding of labels, confusion matrix labeling, and plotting results.

In [ ]:
# ===========================
# Load training data
# ===========================
data1 = h5py.File('Dataset/HisarMod2019.1/Train/train.mat','r')  # Open .mat file in read mode
train = data1['data_save'][:]                                   # Extract dataset from HDF5 group 'data_save'

# Swap axes to match model input shape
# Original shape might be (num_samples, 2, 1024) or (1024, 2, num_samples)
# After swapaxes(0,2), shape becomes (num_samples, 2, 1024)
train = train.swapaxes(0,2)

# ===========================
# Load test data
# ===========================
data2 = h5py.File('Dataset/HisarMod2019.1/Test/test.mat','r')
test = data2['data_save'][:]
test = test.swapaxes(0,2)

# ===========================
# Add channel dimension
# ===========================
# Conv2D layers expect input of shape (samples, height, width, channels)
train = np.expand_dims(train, axis=3)  # Add a channel dimension: shape -> (num_samples, 2, 1024, 1)
test = np.expand_dims(test, axis=3)

In [ ]:
# ===========================
# Load and preprocess labels
# ===========================

# Load training labels from CSV file
train_labels = pd.read_csv('Dataset/HisarMod2019.1/Train/train_labels1.csv', header=None)
train_labels = np.array(train_labels)  # Convert DataFrame to NumPy array

# Convert integer labels to one-hot encoding
# This is required for categorical cross-entropy loss in Keras
train_labels = to_categorical(train_labels, num_classes=None)


# Load test labels from CSV file
test_labels = pd.read_csv('Dataset/HisarMod2019.1/Test/test_labels1.csv', header=None)
test_labels = np.array(test_labels)    # Convert DataFrame to NumPy array
test_labels = to_categorical(test_labels, num_classes=None)  # One-hot encoding


# ===========================
# Load and preprocess SNR values
# ===========================

# Load training SNR values from CSV
# Each row corresponds to the SNR of a training sample
train_snr = pd.read_csv('Dataset/HisarMod2019.1/Train/train_snr.csv', header=None)
train_snr = np.array(train_snr)  # Convert to NumPy array for easier indexing

# Load test SNR values from CSV
# Each row corresponds to the SNR of a test sample
test_snr = pd.read_csv('Dataset/HisarMod2019.1/Test/test_snr.csv', header=None)
test_snr = np.array(test_snr)    # Convert to NumPy array

In [ ]:
# ===========================
# Split dataset into training and validation sets
# ===========================

# Number of total examples in the training dataset
n_examples = train.shape[0]  # train has shape [N, 1024, 2]

# Number of samples for training and validation
n_train = int(n_examples * 0.8)  # 80% for training
n_val = int(n_examples * 0.2)    # 20% for validation

# Randomly select indices for training samples
train_idx = list(np.random.choice(range(0, n_examples), size=n_train, replace=False))

# Remaining indices will be used for validation
val_idx = list(set(range(0, n_examples)) - set(train_idx))

# Shuffle indices to ensure random ordering
np.random.shuffle(train_idx)
np.random.shuffle(val_idx)

# ===========================
# Create datasets using the indices
# ===========================

# Training data and labels
X_train = train[train_idx]         # Shape: [n_train, 1024, 2]
Y_train = train_labels[train_idx]  # Shape: [n_train, num_classes]

# Validation data and labels
X_val = train[val_idx]             # Shape: [n_val, 1024, 2]
Y_val = train_labels[val_idx]      # Shape: [n_val, num_classes]

# Test data and labels (use full test set)
X_test = test                     # Shape: [num_test_samples, 1024, 2]
Y_test = test_labels               # Shape: [num_test_samples, num_classes]

# Test SNR values corresponding to each test sample
Z_test = test_snr                  # Shape: [num_test_samples, 1] or similar

In [ ]:
# ===========================
# Set up training parameters
# ===========================

nb_epoch = 200    # Total number of times the model will iterate over the entire training dataset
batch_size = 300  # Number of samples per gradient update during training


# ===========================
# Create and compile the model
# ===========================

model = ResNet()  # Instantiate the ResNet model defined earlier

# Compile the model with:
# - Loss: categorical_crossentropy (for multi-class classification)
# - Metrics: accuracy (to monitor performance during training)
# - Optimizer: Adam (adaptive learning rate optimizer)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')


# ===========================
# Visualize the model architecture
# ===========================

# Save a graphical representation of the model architecture as 'model_Resnet.png'
# show_shapes=True ensures that the output shapes of each layer are displayed
plot_model(model, to_file='model_Resnet.png', show_shapes=True)
model.summary()

In [ ]:
# ===========================
# Set file path to save the best model weights
# ===========================
filepath = 'weights/weights.h5'  # The model weights will be saved here during training


# ===========================
# Record the start time for training
# ===========================
import time
TRS_ResNet = time.time()  # Timestamp before training begins (used to calculate training duration)


# ===========================
# Train the ResNet model
# ===========================
history = model.fit(
    X_train,                   # Training data
    Y_train,                   # Training labels (one-hot encoded)
    batch_size=batch_size,     # Number of samples per gradient update
    epochs=nb_epoch,           # Total number of passes over the training data
    verbose=2,                 # Verbosity level: 2 = one line per epoch
    validation_data=(X_val, Y_val),  # Data for evaluating validation loss/accuracy after each epoch
    callbacks=[                # List of callback functions to customize training
        # Save the model weights whenever validation loss improves
        keras.callbacks.ModelCheckpoint(
            filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'
        ),
        # Reduce learning rate by factor 0.5 if validation loss plateaus for 5 epochs
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, verbose=1, patince=5, min_lr=0.0000001
        ),
        # Stop training early if validation loss does not improve for 5 epochs
        keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5, verbose=1, mode='auto'
        ),
        # Optional: TensorBoard callback for visualizing training (commented out)
        # keras.callbacks.TensorBoard(histogram_freq=1, write_graph=True, write_images=True)
    ]
)

In [ ]:
# ===========================
# Record end time of training
# ===========================
TRE_ResNet = time.time()  # Timestamp after training completes

# ===========================
# Calculate total training time
# ===========================
T_ResNet = TRE_ResNet - TRS_ResNet  # Total training duration in seconds

In [ ]:
# ===========================
# Evaluate model performance on test set
# ===========================

TES_ResNet = time.time()  # Timestamp before evaluation (can be used to measure inference time)

# Evaluate the trained model on the test data
# - Returns [loss, accuracy] because we compiled the model with loss='categorical_crossentropy' and metrics=['accuracy']
# - verbose=1 prints progress
# - batch_size=batch_size controls how many samples are processed at once during evaluation
score = model.evaluate(X_test, Y_test, verbose=1, batch_size=batch_size)

# Print the evaluation results
# score[0] = test loss, score[1] = test accuracy
print(score)

In [ ]:
calculate_acc_cm_each_snr(Y_test, test_Y_hat, Z_test, classes, min_snr=-18) #plot accuracy

In [ ]:
# ===========================
# Record end time of test evaluation
# ===========================
TEE_ResNet = time.time()  # Timestamp after model evaluation on test set

# ===========================
# Calculate total evaluation/inference time
# ===========================
T_ResNet = TEE_ResNet - TES_ResNet  # Total time taken to evaluate the model on the test data